In [1]:
#importing the libraries we need
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import torchvision
import cv2
from torch.utils.data import Dataset, DataLoader 
from torch.autograd import Variable
from torch import nn 
from utils import *
from opticalFlowGuidedFeatures import OFFModel
from trainOFF import trainOFF

torch.cuda.empty_cache()

ModuleNotFoundError: No module named 'opticalFlowGuidedFeatures'

In [3]:
from torch import nn 

In [ ]:
nn.BatchNorm2d()

In [2]:
import torch

In [ ]:
torch.optim.Adam()

In [2]:
#assigning the type of device that we will work on
device =  torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
#device =  torch.device('cpu')

In [3]:
train_frames = pd.read_csv('frames_train_1.csv')
test_frames = pd.read_csv('frames_test_1.csv')

train_videos = pd.read_csv('video_train_1.csv')
test_videos = pd.read_csv('video_test_1.csv')

In [4]:
train_videos.head()

,name,class,num_frames
0,v_ApplyEyeMakeup_g08_c01.avi,1,24
1,v_ApplyEyeMakeup_g08_c02.avi,1,23
2,v_ApplyEyeMakeup_g08_c03.avi,1,29
3,v_ApplyEyeMakeup_g08_c04.avi,1,44
4,v_ApplyEyeMakeup_g08_c05.avi,1,55


In [ ]:
class Resize(object):
    """Rescale the image to a given size.

    Args:
        output_size (tuple): Desired output size of the image. should be like (new_h , new_w)
    """

    def __init__(self, output_size):

        assert len(output_size) == 2 , "the output size must be of lenght 2 >>> (new_h , new_w)"
        self.output_size = output_size

    def __call__(self, images):

        new_h , new_w = self.output_size

        resized_images = np.zeros( (len(images) , new_h , new_w , 3) )

        for i , image in enumerate(images) :

            resized_images[i] = cv2.resize(src = image, dsize = (new_w,new_h))


        return resized_images


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, images):

        # swap color axis because
        # numpy images: T x H x W x C
        # torch image: C x T X H X W

        images = images.transpose((3, 0, 1, 2))

        return torch.from_numpy(images).float() / 255



In [5]:
seq_len = 16 #the length of the sequence
overlap = 8 #overlap between every two consecutive sequences

train_seqs = [] #intializing the list to hold the sequences of the videos

for i in range(len(train_videos)):
    
    name = train_videos['name'][i]
    
    label = train_videos['class'][i]
    
    numOfFrames = train_videos['num_frames'][i]
    
    seqs = getSequences (numOfFrames , seq_len , overlap = overlap) #get the list of sequences from the current video frames
    
    train_seqs += [(name , label , sequence) for sequence in seqs] 
    
train_seqs = pd.DataFrame(train_seqs , columns = ['name', 'class' ,'seq']) #saving the whole sequences in a datafram

In [6]:
train_seqs.head()

,name,class,seq
0,v_ApplyEyeMakeup_g08_c01.avi,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,v_ApplyEyeMakeup_g08_c01.avi,1,"[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20..."
2,v_ApplyEyeMakeup_g08_c02.avi,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,v_ApplyEyeMakeup_g08_c02.avi,1,"[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20..."
4,v_ApplyEyeMakeup_g08_c03.avi,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [7]:
#validation/train ratio
validation_ratio = 0.1

Image_output_size = (124,124) #output size of the images

train_batch_size = 2
val_batch_size = 1

tr_seqs , val_seqs = vali_train_splite(validation_ratio , train_seqs) #spliting the train_seqs dataframe into train and validation

#the transform steps that we will execute on the images
transform  =torchvision.transforms.Compose([
    Resize(Image_output_size), #resizing the images
    ToTensor() # reshaping and transforming the images to Tensors
])

#intializing the image generator
train_gen = UCF101DatasetSeqs(frame_seq_names =tr_seqs[ : 20 * train_batch_size], img_dir ='Train_Frames_1/', shuffle = True , transform = transform)

#intializing the validation image generator
val_gen = UCF101DatasetSeqs(frame_seq_names =val_seqs, img_dir ='Train_Frames_1/', shuffle = True , transform = transform)

#intializing the train batches loader !!!
train_loader = DataLoader(train_gen , batch_size= train_batch_size , shuffle = True)

#intializing the val batches loader !!!
val_loader = DataLoader(val_gen , batch_size= val_batch_size , shuffle = True)

In [8]:
output_features = 101 #the number of labels

modelV4 = OFFModel(output_features , seq_len , device)   #intializing the model
lossFunc = nn.NLLLoss() #the loss funstion to train on (crossEntropy since it's a classification task)
optimizer = torch.optim.Adam(modelV4.parameters(), lr=0.005) #the optimization algorithm we use during training

epochs = 1000 #num of epoches

In [ ]:
tr_loss_lis, tr_acc_lis, val_loss_lis, val_acc_lis= trainOFF(modelV4 , lossFunc, optimizer , epochs , train_loader ,val_loader , device)

[1, 1] train_loss: 4.63037 train_acc: 0.00000
[1, 10] train_loss: 4.70590 train_acc: 0.00000 val_loss: 4.65516 val_acc: 0.00000
[1, 20] train_loss: 4.64388 train_acc: 0.00000 val_loss: 4.72946 val_acc: 0.00000
[2, 1] train_loss: 4.34405 train_acc: 0.00000
[2, 10] train_loss: 4.23393 train_acc: 10.00000 val_loss: 5.59200 val_acc: 0.00000
[2, 20] train_loss: 4.02357 train_acc: 5.00000 val_loss: 3.54718 val_acc: 0.00000
[3, 1] train_loss: 3.80315 train_acc: 0.00000
[3, 10] train_loss: 3.53039 train_acc: 5.00000 val_loss: 3.81267 val_acc: 0.00000
[3, 20] train_loss: 3.88399 train_acc: 5.00000 val_loss: 7.48435 val_acc: 0.00000
[4, 1] train_loss: 3.33340 train_acc: 0.00000
[4, 10] train_loss: 3.37461 train_acc: 10.00000 val_loss: 7.73524 val_acc: 0.00000
[4, 20] train_loss: 3.79697 train_acc: 0.00000 val_loss: 7.63815 val_acc: 0.00000
[5, 1] train_loss: 3.24145 train_acc: 0.00000
[5, 10] train_loss: 3.42126 train_acc: 0.00000 val_loss: 7.58900 val_acc: 0.00000
[5, 20] train_loss: 3.69553 tr

[77, 10] train_loss: 3.37932 train_acc: 15.00000 val_loss: 10.54724 val_acc: 0.00000
[77, 20] train_loss: 3.50010 train_acc: 0.00000 val_loss: 2.98482 val_acc: 0.00000
[78, 1] train_loss: 3.37495 train_acc: 0.00000
[78, 10] train_loss: 3.33679 train_acc: 15.00000 val_loss: 3.70499 val_acc: 0.00000
[78, 20] train_loss: 3.53092 train_acc: 0.00000 val_loss: 10.05345 val_acc: 0.00000
[79, 1] train_loss: 3.14955 train_acc: 50.00000
[79, 10] train_loss: 3.32923 train_acc: 15.00000 val_loss: 10.39750 val_acc: 0.00000
[79, 20] train_loss: 3.53017 train_acc: 0.00000 val_loss: 3.71657 val_acc: 0.00000
[80, 1] train_loss: 3.34137 train_acc: 0.00000
[80, 10] train_loss: 3.35605 train_acc: 10.00000 val_loss: 10.20129 val_acc: 0.00000
[80, 20] train_loss: 3.49957 train_acc: 5.00000 val_loss: 3.66997 val_acc: 0.00000
[81, 1] train_loss: 3.22588 train_acc: 50.00000
[81, 10] train_loss: 3.29681 train_acc: 10.00000 val_loss: 10.39155 val_acc: 0.00000
[81, 20] train_loss: 3.57110 train_acc: 5.00000 val_l

[152, 10] train_loss: 3.36809 train_acc: 5.00000 val_loss: 3.69960 val_acc: 0.00000
[152, 20] train_loss: 3.47259 train_acc: 10.00000 val_loss: 3.68591 val_acc: 0.00000
[153, 1] train_loss: 3.35626 train_acc: 0.00000
[153, 10] train_loss: 3.43852 train_acc: 5.00000 val_loss: 3.62542 val_acc: 0.00000
[153, 20] train_loss: 3.40267 train_acc: 10.00000 val_loss: 11.03382 val_acc: 0.00000
[154, 1] train_loss: 3.33494 train_acc: 0.00000
[154, 10] train_loss: 3.38765 train_acc: 10.00000 val_loss: 10.70421 val_acc: 0.00000
[154, 20] train_loss: 3.45663 train_acc: 5.00000 val_loss: 10.89384 val_acc: 0.00000
[155, 1] train_loss: 3.65700 train_acc: 0.00000
[155, 10] train_loss: 3.51783 train_acc: 0.00000 val_loss: 10.77605 val_acc: 0.00000
[155, 20] train_loss: 3.32346 train_acc: 15.00000 val_loss: 10.79727 val_acc: 0.00000
[156, 1] train_loss: 3.01395 train_acc: 0.00000
[156, 10] train_loss: 3.45472 train_acc: 10.00000 val_loss: 11.56417 val_acc: 0.00000
[156, 20] train_loss: 3.39037 train_acc: 

[227, 1] train_loss: 3.64914 train_acc: 0.00000
[227, 10] train_loss: 3.30025 train_acc: 15.00000 val_loss: 3.68773 val_acc: 0.00000
[227, 20] train_loss: 3.53455 train_acc: 0.00000 val_loss: 12.26456 val_acc: 0.00000
[228, 1] train_loss: 3.36420 train_acc: 0.00000
[228, 10] train_loss: 3.46831 train_acc: 5.00000 val_loss: 11.40241 val_acc: 0.00000
[228, 20] train_loss: 3.36878 train_acc: 10.00000 val_loss: 3.66751 val_acc: 0.00000
[229, 1] train_loss: 3.65754 train_acc: 0.00000
[229, 10] train_loss: 3.37014 train_acc: 5.00000 val_loss: 12.36695 val_acc: 0.00000
[229, 20] train_loss: 3.46248 train_acc: 10.00000 val_loss: 3.67918 val_acc: 0.00000
[230, 1] train_loss: 3.68716 train_acc: 0.00000
[230, 10] train_loss: 3.34777 train_acc: 10.00000 val_loss: 12.40918 val_acc: 0.00000
[230, 20] train_loss: 3.48235 train_acc: 5.00000 val_loss: 3.67217 val_acc: 0.00000
[231, 1] train_loss: 3.14532 train_acc: 50.00000
[231, 10] train_loss: 3.46568 train_acc: 5.00000 val_loss: 11.87939 val_acc: 0.

[301, 20] train_loss: 3.47995 train_acc: 5.00000 val_loss: 12.73561 val_acc: 0.00000
[302, 1] train_loss: 2.81869 train_acc: 50.00000
[302, 10] train_loss: 3.40563 train_acc: 10.00000 val_loss: 2.59720 val_acc: 100.00000
[302, 20] train_loss: 3.41503 train_acc: 5.00000 val_loss: 3.00873 val_acc: 0.00000
[303, 1] train_loss: 3.35967 train_acc: 0.00000
[303, 10] train_loss: 3.44976 train_acc: 0.00000 val_loss: 3.02422 val_acc: 0.00000
[303, 20] train_loss: 3.37239 train_acc: 15.00000 val_loss: 12.13050 val_acc: 0.00000
[304, 1] train_loss: 3.33513 train_acc: 0.00000
[304, 10] train_loss: 3.41256 train_acc: 10.00000 val_loss: 12.50579 val_acc: 0.00000
[304, 20] train_loss: 3.41368 train_acc: 5.00000 val_loss: 12.69517 val_acc: 0.00000
[305, 1] train_loss: 3.34902 train_acc: 0.00000
[305, 10] train_loss: 3.30890 train_acc: 10.00000 val_loss: 12.88186 val_acc: 0.00000
[305, 20] train_loss: 3.51227 train_acc: 5.00000 val_loss: 3.66978 val_acc: 0.00000
[306, 1] train_loss: 3.65463 train_acc: 

[376, 10] train_loss: 3.43013 train_acc: 5.00000 val_loss: 3.66785 val_acc: 0.00000
[376, 20] train_loss: 3.38664 train_acc: 10.00000 val_loss: 3.67885 val_acc: 0.00000
[377, 1] train_loss: 3.67567 train_acc: 0.00000
[377, 10] train_loss: 3.42839 train_acc: 5.00000 val_loss: 3.67334 val_acc: 0.00000
[377, 20] train_loss: 3.38975 train_acc: 10.00000 val_loss: 3.00507 val_acc: 0.00000
[378, 1] train_loss: 3.34107 train_acc: 0.00000
[378, 10] train_loss: 3.40922 train_acc: 10.00000 val_loss: 3.68265 val_acc: 0.00000
[378, 20] train_loss: 3.40635 train_acc: 5.00000 val_loss: 3.66400 val_acc: 0.00000
[379, 1] train_loss: 3.67076 train_acc: 0.00000
[379, 10] train_loss: 3.54551 train_acc: 0.00000 val_loss: 12.17591 val_acc: 0.00000
[379, 20] train_loss: 3.27783 train_acc: 15.00000 val_loss: 12.26094 val_acc: 0.00000
[380, 1] train_loss: 3.05182 train_acc: 0.00000
[380, 10] train_loss: 3.33550 train_acc: 5.00000 val_loss: 13.99196 val_acc: 0.00000
[380, 20] train_loss: 3.49202 train_acc: 10.0

In [40]:
a = (torch.max(torch.randn(16,101) , 1)[1] == 50 )

In [41]:
a

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False])

In [39]:
a

tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False])

In [37]:
a.shape

torch.Size([16])

In [34]:
len(a)

1

In [32]:
len(a)

16

In [29]:
len(a)

16

In [26]:
a

tensor(False)

In [13]:
train_gen[0]['image'].shape

torch.Size([3, 16, 124, 124])

In [10]:
itere = iter(train_loader)

In [11]:
a = next(itere)

In [12]:
a['image'].shape

torch.Size([16, 3, 16, 124, 124])

In [17]:
a['image'].transpose(1,2).reshape(-1 , 3 , 124 , 124).shape

torch.Size([256, 3, 124, 124])

In [13]:
a['image'].transpose(1,2).view (-1 , 3 , 124 , 124).shape

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [19]:
a = torch.randn(16, 3, 16, 124, 124)

In [20]:
a.shape

torch.Size([16, 3, 16, 124, 124])

In [26]:
a.transpose(1,2).shape

torch.Size([16, 16, 3, 124, 124])

In [4]:

model = OFFModel(101, 2 , 16)

In [ ]:
def trainOFF (model , lossFunc, optimizer , epochs , train_loader , validation_loader , device):
    
    tr_loss_lis = [] 
    tr_acc_lis = []
    
    val_loss_lis = [] 
    val_acc_lis = []
    
    val_load_iter = iter(validation_loader)
    
    
    for epoch in range (epochs): # looping over the epochs
        
        tr_running_loss = 0 
        tr_tot_acc = 0
        
        
        for i , data in enumerate(train_loader):
            
            model.train()
            
            images = data['image'] #shape (batch , seq , 3 ,124 ,124)
            labels = data['label']
            
            labels -= 1
            
            images = Variable(images.view (-1 , 3 , 124 , 124)) #shape (batch * seq , 3 ,124 ,124)
            labels = Variable(labels , requires_grad=False)
            

            optimizer.zero_grad()
            
            outputs = model(images.to(device)) #forward + backward + optimization
            
            loss = lossFunc(outputs , labels)
            
            loss.backward()
            
            optimizer.step()
            
            tr_running_loss += loss.item()
            
            
            
            with torch.no_grad():
                
                _ , predicted = torch.max(outputs, 1)
                c = (predicted == labels).squeeze()
                tr_acc = (c.sum() / len(c)).item()
                tr_tot_acc  += tr_acc
                
                
                
                
            
            if (i+1)% 10 == 0  :
                
                model.eval()
                
                with torch.no_grad():
                
                    try :

                        val_data = next (val_load_iter)


                    except :

                        val_load_iter = iter(validation_loader)
                        val_data = next (val_load_iter)



                    images_val = val_data['image']
                    labels_val = val_data['label']

                    labels_val -= 1
                    
                    images_val = Variable(images_val.view (-1 , 3 , 124 , 124) , requires_grad=False) #shape (batch * seq , 3 ,124 ,124)
                    labels_val = Variable(labels_val , requires_grad=False)
                    
                    outputs_val = model(images_val.to(device)) #forward only

                    loss_val = lossFunc(outputs_val , labels_val).item()
                

                    _ , predicted_val = torch.max(outputs_val, 1)
                    c = (predicted_val == labels_val).squeeze()
                    val_acc = (c.sum() / len(c)).item()
                    
                
                    
                
                print(f'[{epoch+1}, {i+1}] train_loss:{tr_running_loss / 10 : .5f} train_acc:{100 * tr_tot_acc / 10 : 0.5f} val_loss:{loss_val : .5f} val_acc:{100 * val_acc : 0.5f}' )
                
                tr_loss_lis.append(tr_running_loss)
                tr_acc_lis.append(tr_tot_acc)
                
                val_loss_lis.append(loss_val)
                val_acc_lis.append(val_acc)
                
                tr_tot_acc = 0
                tr_running_loss = 0.0
                
                
                
            if i == 0 :
                
                print(f'[{epoch+1}, {i+1}] train_loss:{tr_running_loss : .5f} train_acc:{100 * tr_tot_acc : 0.5f}' )
                
                tr_loss_lis.append(tr_running_loss)
                tr_acc_lis.append(tr_tot_acc)
                     
        
    print('Finished Runing')
    
    return tr_loss_lis, tr_acc_lis, val_loss_lis, val_acc_lis